In [0]:
!pip install icrawler

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
from icrawler.builtin import GoogleImageCrawler

brands = ['Fender', 'Gibson', 'Ibanez', 'Caparison', 'Jackson', 
          'DEAN', 'B.C. Rich', 'ESP', 'SCHECTER', 'Paul Reed Smith', 
          'Mayones','Strandberg', 'Kiesel', 'Suhr']

for brand in brands:
  os.makedirs('./data/{}'.format(brand), exist_ok=True)
  crawler = GoogleImageCrawler(storage={"root_dir": "./data/{}".format(brand)})
  crawler.crawl(keyword="{} guitar".format(brand), max_num=500)

In [0]:
%cd /content/gdrive/My Drive/Colab Notebooks/personal_project/guitar_brand_recognition

/content/gdrive/My Drive/Colab Notebooks/personal_project/guitar_brand_recognition


In [0]:
from pathlib import Path
import shutil
import numpy as np

np.random.seed(0)
test_size = 0.2

p = Path('.')

for brand in brands:
    # 元画像ディレクトリのパスオブジェクト
    raw_dir = p / 'data' / brand
    
    # 訓練データを格納するディレクトリ作成
    train_set = p / 'train_set' / brand
    train_set.mkdir(parents=True, exist_ok=True)
    
    # テストデータを格納するディレクトリ作成
    test_set = p / 'test_set' / brand
    test_set.mkdir(parents=True, exist_ok=True)
    
     # 各ブランドのの画像一覧取得
    imgs_path = list(raw_dir.iterdir())
    
    # 訓練データを格納
    for img in imgs_path[:int(len(imgs_path)*0.2)]:
        # 訓練画像をコピー
        shutil.copy(str(img), str(test_set.resolve()))
        
    # テストデータを格納
    for img in imgs_path[int(len(imgs_path)*0.2):]:
        # 訓練画像をコピー
        shutil.copy(str(img), str(train_set.resolve()))

In [0]:
path = Path('./train_set')

num_imgs = 0
for brand in path.iterdir():
  for img in brand.iterdir():
    num_imgs += 1
    
num_imgs

5005

In [0]:
path = Path('./test_set')

num_imgs = 0
for brand in path.iterdir():
  for img in brand.iterdir():
    num_imgs += 1
    
num_imgs

1046

In [0]:
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, Input
from keras import optimizers
from keras import regularizers
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import EarlyStopping 

IMAGE_SIZE = 224
BATCH_SIZE = 32

NUM_TRAINING = 5005
NUM_VALIDATION = 1046

input_tensor = Input(shape=(224, 224, 3))
inceptionv3_model =InceptionV3(weights='imagenet', include_top=False, 
                         input_tensor=input_tensor)

x = inceptionv3_model.output
x = Flatten()(x)
predictions = Dense(14, activation='softmax',kernel_regularizer=regularizers.l2(0.01), name='predictions')(x)
model = Model(inputs=inceptionv3_model.input, outputs=predictions)

for layer in inceptionv3_model.layers[:249]:
  layer.trainable = False
  
  if layer.name.startswith('batch_normalization'):
        layer.trainable = True

for layer in inceptionv3_model.layers[249:]:
  layer.trainable = True

model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1.0 / 255
)

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255
)

train_generator = train_datagen.flow_from_directory(
    'train_set',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
   'test_set',
   target_size=(IMAGE_SIZE, IMAGE_SIZE),
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True
)

hist = model.fit_generator(
    train_generator,
    steps_per_epoch=NUM_TRAINING // BATCH_SIZE,
    epochs=32,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=NUM_VALIDATION//BATCH_SIZE
)

model.save('model.h5')


Found 4992 images belonging to 14 classes.
Found 1044 images belonging to 14 classes.
Epoch 1/32


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


156/156 [==============================] - 213s 1s/step - loss: 3.7225 - acc: 0.2111 - val_loss: 2.5769 - val_acc: 0.3506
Epoch 2/32
156/156 [==============================] - 183s 1s/step - loss: 2.2404 - acc: 0.3584 - val_loss: 1.8086 - val_acc: 0.4990
Epoch 3/32
156/156 [==============================] - 184s 1s/step - loss: 1.8689 - acc: 0.4535 - val_loss: 1.7391 - val_acc: 0.5909
Epoch 4/32
156/156 [==============================] - 184s 1s/step - loss: 1.6368 - acc: 0.5172 - val_loss: 1.3490 - val_acc: 0.6443
Epoch 5/32
156/156 [==============================] - 182s 1s/step - loss: 1.4292 - acc: 0.5783 - val_loss: 1.1855 - val_acc: 0.6848
Epoch 6/32
156/156 [==============================] - 184s 1s/step - loss: 1.2882 - acc: 0.6264 - val_loss: 1.1275 - val_acc: 0.6966
Epoch 7/32
156/156 [==============================] - 183s 1s/step - loss: 1.1925 - acc: 0.6595 - val_loss: 0.9743 - val_acc: 0.7441
Epoch 8/32
156/156 [==============================] - 183s 1s/step - loss: 1.033

In [0]:
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, Input
from keras import optimizers
from keras import regularizers
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import EarlyStopping 

IMAGE_SIZE = 224
BATCH_SIZE = 32

NUM_TRAINING = 5005
NUM_VALIDATION = 1046

input_tensor = Input(shape=(224, 224, 3))
inceptionv3_model =InceptionV3(weights='imagenet', include_top=False, 
                         input_tensor=input_tensor)

x = inceptionv3_model.output
x = Flatten()(x)
predictions = Dense(14, activation='softmax',kernel_regularizer=regularizers.l2(0.01), name='predictions')(x)
model = Model(inputs=inceptionv3_model.input, outputs=predictions)

for layer in inceptionv3_model.layers[:249]:
  layer.trainable = False
  
  if layer.name.startswith('batch_normalization'):
        layer.trainable = True

for layer in inceptionv3_model.layers[249:]:
  layer.trainable = True

model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1.0 / 255
)

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255
)

train_generator = train_datagen.flow_from_directory(
    'train_set',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
   'test_set',
   target_size=(IMAGE_SIZE, IMAGE_SIZE),
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True
)

hist = model.fit_generator(
    train_generator,
    steps_per_epoch=NUM_TRAINING // BATCH_SIZE,
    epochs=64,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=NUM_VALIDATION//BATCH_SIZE
)

model.save('model_1.h5')


Found 4992 images belonging to 14 classes.
Found 1044 images belonging to 14 classes.
Epoch 1/64


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


156/156 [==============================] - 197s 1s/step - loss: 3.9917 - acc: 0.2252 - val_loss: 3.2390 - val_acc: 0.3613
Epoch 2/64
156/156 [==============================] - 176s 1s/step - loss: 2.3971 - acc: 0.3518 - val_loss: 1.6853 - val_acc: 0.5435
Epoch 3/64
156/156 [==============================] - 171s 1s/step - loss: 1.8947 - acc: 0.4437 - val_loss: 1.5899 - val_acc: 0.5623
Epoch 4/64
156/156 [==============================] - 172s 1s/step - loss: 1.7549 - acc: 0.4900 - val_loss: 1.3290 - val_acc: 0.6156
Epoch 5/64
156/156 [==============================] - 172s 1s/step - loss: 1.4825 - acc: 0.5647 - val_loss: 1.2097 - val_acc: 0.6571
Epoch 6/64
156/156 [==============================] - 173s 1s/step - loss: 1.3010 - acc: 0.6206 - val_loss: 1.0468 - val_acc: 0.6897
Epoch 7/64
156/156 [==============================] - 171s 1s/step - loss: 1.1756 - acc: 0.6587 - val_loss: 0.8440 - val_acc: 0.7796
Epoch 8/64
156/156 [==============================] - 171s 1s/step - loss: 1.042

In [0]:
from keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = 224
BATCH_SIZE = 32
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1.0 / 255
)

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255
)

train_generator = train_datagen.flow_from_directory(
    'train_set',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

train_generator.class_indices

Found 4992 images belonging to 14 classes.


{'B.C. Rich': 0,
 'Caparison': 1,
 'DEAN': 2,
 'ESP': 3,
 'Fender': 4,
 'Gibson': 5,
 'Ibanez': 6,
 'Jackson': 7,
 'Kiesel': 8,
 'Mayones': 9,
 'Paul Reed Smith': 10,
 'SCHECTER': 11,
 'Strandberg': 12,
 'Suhr': 13}

In [0]:
from keras.models import load_model

model = load_model('model_1.h5')

Using TensorFlow backend.


In [0]:
from PIL import Image
import numpy as np
import time
img = Image.open('05-313457100_2.jpg')

img_resize = img.resize((224, 224))

img_array = np.asarray(img_resize)
img_array = img_array / 255.
img_array = img_array.reshape((1, 224, 224, 3))

t1 = time.time()
prediction = model.predict(img_array)
print(np.argmax(prediction))
t2 = time.time()
print(t2 - t1)

4
1.637706995010376


In [0]:
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, Input
from keras import optimizers
from keras import regularizers
from keras.applications.mobilenet import MobileNet
from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import EarlyStopping 

IMAGE_SIZE = 224
BATCH_SIZE = 32

NUM_TRAINING = 5005
NUM_VALIDATION = 1046

input_tensor = Input(shape=(224, 224, 3))
base_model =MobileNet(weights='imagenet', include_top=False, 
                         input_tensor=input_tensor)

x = base_model.output
x = Flatten()(x)
predictions = Dense(14, activation='softmax',kernel_regularizer=regularizers.l2(0.01), name='predictions')(x)
model = Model(base_model.input, outputs=predictions)

# for layer in model.layers[:72]:
#     layer.trainable = False

#     # Batch Normalization の freeze解除
#     if "bn" in layer.name:
#         layer.trainable = True

# #73層以降、学習させる
# for layer in model.layers[72:]:
#     layer.trainable = True

model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1.0 / 255
)

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255
)

train_generator = train_datagen.flow_from_directory(
    'train_set',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
   'test_set',
   target_size=(IMAGE_SIZE, IMAGE_SIZE),
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True
)

hist = model.fit_generator(
    train_generator,
    steps_per_epoch=NUM_TRAINING // BATCH_SIZE,
    epochs=64,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=NUM_VALIDATION//BATCH_SIZE
)

model.save('mobilenet_model.h5')


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Found 4992 images belonging to 14 classes.
Found 1044 images belonging to 14 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/64
  1/156 [..............................] - ETA: 53:29 - loss: 5.5880 - acc: 0.0625

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


156/156 [==============================] - 1734s 11s/step - loss: 14.6317 - acc: 0.0994 - val_loss: 14.7971 - val_acc: 0.0957
Epoch 2/64
156/156 [==============================] - 164s 1s/step - loss: 8.4381 - acc: 0.1072 - val_loss: 15.2344 - val_acc: 0.0741
Epoch 3/64
156/156 [==============================] - 167s 1s/step - loss: 3.3402 - acc: 0.1202 - val_loss: 9.7309 - val_acc: 0.0543
Epoch 4/64
156/156 [==============================] - 169s 1s/step - loss: 2.7459 - acc: 0.1583 - val_loss: 3.0314 - val_acc: 0.1136
Epoch 5/64
156/156 [==============================] - 167s 1s/step - loss: 2.6183 - acc: 0.1697 - val_loss: 2.8384 - val_acc: 0.1275
Epoch 6/64
156/156 [==============================] - 169s 1s/step - loss: 2.5302 - acc: 0.1833 - val_loss: 2.7674 - val_acc: 0.1225
Epoch 7/64
156/156 [==============================] - 168s 1s/step - loss: 2.4656 - acc: 0.1975 - val_loss: 2.8520 - val_acc: 0.1749
Epoch 8/64
156/156 [==============================] - 168s 1s/step - loss: 